In [2]:
from cmath import exp

from ucimlrepo import fetch_ucirepo
import numpy as np
from sklearn.model_selection import train_test_split

heart_disease = fetch_ucirepo(id=45)
X = heart_disease.data.features
Y = heart_disease.data.targets
X.fillna({'ca': X['ca'].median(), 'thal': X['thal'].mode()[0]}, inplace=True)

for index, row in Y.iterrows():
    if row[0] != 0:
        row[0] = 1 #jeżeli nie ma klasy 0 to ma klasę 1

y = Y.to_numpy()
X = (X-X.min())/(X.max()-X.min())
x = X.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2)

c:\users\krzys\desktop\pythonproject\againterpreter\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [3]:
from random import seed
from random import uniform

# def initialize_network(n_inputs, n_hidden, n_outputs):
# 	network = list()
# 	# +1 jest bo bias
# 	hidden_layer = [{'weights':[uniform(1, 100) for i in range(n_inputs + 1)]} for i in range(n_hidden)]
# 	network.append(hidden_layer)
# 	output_layer = [{'weights':[uniform(1, 100) for i in range(n_hidden + 1)]} for i in range(n_outputs)]
# 	network.append(output_layer)
# 	return network

# def initialize_network(n_inputs, hidden_layers, n_outputs):
#     network = []
#
#     for num_neurons in hidden_layers:
#         hidden_layer = [{'weights': [uniform(1, 100) for i in range(n_inputs + 1)]} for i in range(num_neurons)]
#         network.append(hidden_layer)
#         n_inputs = num_neurons
#
#     output_layer = [{'weights': [uniform(1, 100) for i in range(n_inputs + 1)]} for i in range(n_outputs)]
#     network.append(output_layer)
#
#     return network
def initialize_network(n_inputs, hidden_layers, n_outputs, std_dev, mean):
    network = []
    for num_neurons in hidden_layers:
        hidden_layer = [{'weights': list(np.random.normal(mean, std_dev, n_inputs + 1))} for i in range(num_neurons)]
        network.append(hidden_layer)
        n_inputs = num_neurons

    output_layer = [{'weights': list(np.random.normal(mean, std_dev, n_inputs + 1))} for i in range(n_outputs)]
    network.append(output_layer)

    return network


seed(1)
n_inputs = 13
hidden_layers = [4,2]
n_outputs = 1
std_deviation = 0.1
network = initialize_network(n_inputs, hidden_layers, n_outputs, std_deviation, 50)
# network = initialize_network(13, [4, 3], 1, [0.1, 0.2])
for layer in network:
	print(layer) # tutaj jest o 1 więcej wag bo bias
	print()

[{'weights': [49.817166174553414, 50.20830670613731, 49.99049552151656, 49.909653399882906, 50.02847021805165, 49.86938639292833, 49.94778162916356, 49.831188148707064, 50.112118078879774, 50.07972260889731, 50.02660056463222, 49.92605749459056, 49.94957701380815, 50.02114314862527]}, {'weights': [50.06322138801076, 50.22832032814158, 49.903662577820064, 50.12324676104338, 49.89283525146773, 49.998338283517825, 50.11154893234289, 49.939122332861366, 49.852334298456725, 50.025859942906415, 50.327921316915806, 50.17773789818495, 50.01855053470606, 50.0429387316317]}, {'weights': [49.99236655850847, 49.933634794609475, 50.15157120950434, 49.91448946540333, 49.908432939015285, 49.93521955535791, 49.92992983460995, 50.032289288888215, 50.00980561334021, 50.06759532107658, 49.89296352594946, 50.11217400175912, 49.856436774621756, 49.892720325047996]}, {'weights': [50.05186548312849, 49.98090832343655, 49.88558163743578, 50.10857790579012, 49.81972168361326, 50.01564521483046, 49.967222096877

In [4]:
#ostatnia waga jest biasem
def activate(weights, inputs):
	activation = weights[-1]	#biasa nie mnożymy
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]	#sumujemy argument do sigmoida
	return activation

def transfer(activation):
	return 1.0 / (1.0 + exp(-activation/200))	# sigmoid

# pochodna sigmoida
def transfer_derivative(output):
	return (output/200) * (1.0 - output/200)

def loss_fun(y, y_pred):
    epsilon = 1e-15  # zapobieganie log(0)
    loss = - (y * np.log(y_pred + epsilon) + (1 - y) * np.log(1 - y_pred + epsilon))

    return loss

In [5]:
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:	# bierzemy wagi danego neuronu
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)	# wartość wyjściowa neurona zapisana na potem
			new_inputs.append(neuron['output'])
		inputs = new_inputs		# przechodzi jako nowy wkład kolejnej warstwy
	return inputs

In [6]:
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))): # idziemy od końca
		layer = network[i]
		errors = list()
		#jeżeli to nie jest ostatnia warstwa (zaczynamy i tak od ostatniej)
		if i != len(network)-1:
			for j in range(len(layer)):	#przechodzimy po neuronach warstwy
				error = 0.0
				for neuron in network[i + 1]:
					# pobieramy wagi łączące ten neuron z prawym (z tablicy neurona następnej warstwy) i zapisujemy error
					# ponieważ my tutaj jesteśmy w poprzedniej warstwie (bardziej prawej) to
					# neuron delta już jest policzony
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		#jeżeli to ostatnia (czyli pierwsza) warstwa to liczę błąd
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		#obliczenia w danej warstwie, przechodzimy ją aktualizując wartości błędu neuronu
		# przy pomocy pochodnej
		for j in range(len(layer)):
			neuron = layer[j]
			# delta = błąd dla każdego neuronu razy pochodna
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [14]:
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1] # bo ostatni jest bias
		if i != 0:	#jeżeli to nie jest pierwsza warstwa updatujemy wagi od lewej teraz
			#output poprzedniej warstwy to input
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				#aktualizacja wag w tablicy neuronu danej warstwy
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			#aktualizacja biasu
			neuron['weights'][-1] -= l_rate * neuron['delta']


def train_network(network, train, l_rate, n_epoch, n_outputs):
	avg_loss = 0
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			expected = [0 for i in range(n_outputs)]
			expected[0] = int(row[-1])
			outputs = forward_propagate(network, row)
			sum_error += loss_fun(int(row[-1]), outputs[0]) #na sztywno pierwszy element żeby nie komplikować
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

		avg_loss = sum_error/len(train)
		print(f"Epoka: {epoch}, avg_loss: {avg_loss}")

	return avg_loss

In [15]:
num_iterations = 20  # Adjust this as needed
best_mean = None
best_std_deviation = None
best_loss = float('inf')
training_data = np.hstack((x_train, y_train.astype(int))).tolist()

for _ in range(num_iterations):
    # losowe wartości dla mean i odchylenia
    mean = np.random.uniform(5, 150)
    std_deviation = np.random.uniform(2, 50)

    network = initialize_network(n_inputs, hidden_layers, n_outputs, std_deviation, mean)
    output_loss = train_network(network, training_data, 0.9, 50, 1)

    if output_loss < best_loss:
        best_mean = mean
        best_std_deviation = std_deviation
        best_loss = output_loss

print("Best Mean:", best_mean)
print("Best Std Deviation:", best_std_deviation)
print("Best Average Loss:", best_loss)

Epoka: 0, avg_loss: (1.4283482611413534+0j)
Epoka: 1, avg_loss: (1.4249216277525745+0j)
Epoka: 2, avg_loss: (1.421504525920841+0j)
Epoka: 3, avg_loss: (1.4180970098096721+0j)
Epoka: 4, avg_loss: (1.4146991332262921+0j)
Epoka: 5, avg_loss: (1.411310949603949+0j)
Epoka: 6, avg_loss: (1.4079325119842576+0j)
Epoka: 7, avg_loss: (1.4045638729996315+0j)
Epoka: 8, avg_loss: (1.4012050848557889+0j)
Epoka: 9, avg_loss: (1.3978561993143432+0j)
Epoka: 10, avg_loss: (1.3945172676755215+0j)
Epoka: 11, avg_loss: (1.3911883407609975+0j)
Epoka: 12, avg_loss: (1.3878694688968805+0j)
Epoka: 13, avg_loss: (1.3845607018968484+0j)
Epoka: 14, avg_loss: (1.3812620890454688+0j)
Epoka: 15, avg_loss: (1.3779736790816899+0j)
Epoka: 16, avg_loss: (1.3746955201825597+0j)
Epoka: 17, avg_loss: (1.3714276599471364+0j)
Epoka: 18, avg_loss: (1.3681701453806512+0j)
Epoka: 19, avg_loss: (1.3649230228789093+0j)
Epoka: 20, avg_loss: (1.3616863382129418+0j)
Epoka: 21, avg_loss: (1.358460136513941+0j)
Epoka: 22, avg_loss: (1

In [18]:
seed(1)
training_data = np.hstack((x_train, y_train.astype(int))).tolist()
n_inputs = len(training_data[0]) - 1

n_inputs = 13
hidden_layers = [5,5,3]
n_outputs = 1
std_deviation = 41.78
mean = 8.801
network = initialize_network(n_inputs, hidden_layers, n_outputs, std_deviation, mean)

# network = initialize_network(n_inputs, [5,5,5], 1)
output_data = train_network(network, training_data, 0.5, 50, 1)

Epoka: 0, avg_loss: (0.7194916328804951+0j)
Epoka: 1, avg_loss: (0.7194343672045368+0j)
Epoka: 2, avg_loss: (0.7193772226732709+0j)
Epoka: 3, avg_loss: (0.7193201990002579+0j)
Epoka: 4, avg_loss: (0.7192632958998205+0j)
Epoka: 5, avg_loss: (0.7192065130870355+0j)
Epoka: 6, avg_loss: (0.7191498502777377+0j)
Epoka: 7, avg_loss: (0.7190933071885155+0j)
Epoka: 8, avg_loss: (0.7190368835367056+0j)
Epoka: 9, avg_loss: (0.718980579040395+0j)
Epoka: 10, avg_loss: (0.7189243934184183+0j)
Epoka: 11, avg_loss: (0.7188683263903519+0j)
Epoka: 12, avg_loss: (0.7188123776765174+0j)
Epoka: 13, avg_loss: (0.7187565469979719+0j)
Epoka: 14, avg_loss: (0.7187008340765152+0j)
Epoka: 15, avg_loss: (0.7186452386346794+0j)
Epoka: 16, avg_loss: (0.718589760395731+0j)
Epoka: 17, avg_loss: (0.7185343990836675+0j)
Epoka: 18, avg_loss: (0.7184791544232184+0j)
Epoka: 19, avg_loss: (0.7184240261398339+0j)
Epoka: 20, avg_loss: (0.7183690139596949+0j)
Epoka: 21, avg_loss: (0.718314117609703+0j)
Epoka: 22, avg_loss: (0

In [22]:
def min_max_scale(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

def predict(data):
	outputs = []
	y_preds = []

	for row in data:
		output = forward_propagate(network, row)[0].real
		outputs.append(output)

	min_output = min(outputs)
	max_output = max(outputs)

	for output in outputs:
		scaled_output = min_max_scale(output, min_output, max_output)
		# print(scaled_output)
		binary_prediction = 1 if scaled_output >= 0.5 else 0
		y_preds.append(binary_prediction)

	return np.array(y_preds)

# def predict(data):
# 	outputs = []
# 	y_preds = []
# 	for row in data:
# 		output = forward_propagate(network, row)[0].real
# 		outputs.append(output)
# 		# print(output)
# 		binary_prediction = 1 if output >= 0.65 else 0
# 		y_preds.append(binary_prediction)
#
# 	return np.array(y_preds)
#Ocena modelu
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_pred = predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(f'Confusion: {confusion}')
print(f'Report: {report}')

Accuracy: 0.7213114754098361
Confusion: [[24  9]
 [ 8 20]]
Report:               precision    recall  f1-score   support

           0       0.75      0.73      0.74        33
           1       0.69      0.71      0.70        28

    accuracy                           0.72        61
   macro avg       0.72      0.72      0.72        61
weighted avg       0.72      0.72      0.72        61

